In [1]:
import sys
sys.path.append('/local_data/kni/_branch_examine/verify_optim/verify_refactor/')
from attalos.imgtxt_algorithms.main import *

In [2]:
# # Temp object using duck typing to replace command line arguments
args = lambda: None

# args.image_feature_file_train = ""
# args.text_feature_file_train = ""
# args.image_feature_file_test = ""
# args.text_feature_file_test = ""

args.image_feature_file_train = "/local_data/teams/attalos/features/image/iaprtc_train_20160816_inception.hdf5"
args.text_feature_file_train = "/local_data/teams/attalos/features/text/iaprtc_train_20160816_text.json.gz"
args.image_feature_file_test = "/local_data/teams/attalos/features/image/iaprtc_test_20160816_inception.hdf5"
args.text_feature_file_test = "/local_data/teams/attalos/features/text/iaprtc_test_20160816_text.json.gz"
# args.image_feature_file_test = "/local_data/teams/attalos/features/image/espgame_test_20160823_inception.hdf5"
# args.text_feature_file_test = "/local_data/teams/attalos/features/text/espgame_test_20160823_text.json.gz"


# args.word_vector_file = ""
# args.word_vector_type = "glove" # "w2v"
args.word_vector_file = "/local_data/kni/data/vectors-phrase.bin"
args.word_vector_type = "w2v"

args.model_type = 'negsampling' # "yfccjoint" #negsamp" #"wdv" #"naivesum" #"multihot"
args.cross_eval = False
args.in_memory = True
args.model_input_path = None
args.model_output_path = None
args.num_epochs = 500
args.batch_size = 256
args.learning_rate = 0.00005
args.verbose_eval = True
args.epoch_verbosity = 1
args.optim_words = True
args.hidden_units='2048,1024'

In [3]:
logger.info("Parsing train and test datasets.")
train_dataset = Dataset(args.image_feature_file_train, args.text_feature_file_train, load_image_feats_in_mem=args.in_memory)
test_dataset = Dataset(args.image_feature_file_test, args.text_feature_file_test)

[2016-09-27 19:43:53,629] [INFO] Parsing train and test datasets.


In [4]:
logger.info("Reading word vectors from file.")
wv_model = load_wv_model(args.word_vector_file, args.word_vector_type)

[2016-09-27 19:43:53,903] [INFO] Reading word vectors from file.


In [5]:
#sess.close()

In [6]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)

In [7]:
model_cls = ModelTypes[args.model_type].value
logger.info("Selecting model class: %s" % model_cls.__name__)
datasets = [train_dataset] if args.cross_eval else [train_dataset, test_dataset]
model = model_cls(wv_model, datasets, **vars(args))
model.initialize_model(sess)

[2016-09-27 19:44:07,365] [INFO] Selecting model class: NegSamplingModel
[2016-09-27 19:44:08,537] [INFO] Optimization on GPU, word vectors are stored separately.
[2016-09-27 19:44:08,579] [INFO] Optimization uses Adam


In [8]:
model.Cwd.shape

(288, 168)

In [ ]:
logger.info("Preparing test_dataset.")
fetches, feed_dict, truth = model.prep_predict(test_dataset)

[2016-09-27 19:44:10,321] [INFO] Preparing test_dataset.


In [ ]:
logger.info("Starting training phase.")
mid_eval = (fetches, feed_dict, truth) if args.verbose_eval else None
train(sess, model, args.num_epochs, train_dataset, args.batch_size, args.epoch_verbosity, mid_eval=mid_eval) 
#, train_dataset, wv_model, test_dataset=test_dataset, epoch_verbosity_rate=100)

[2016-09-27 19:44:10,712] [INFO] Starting training phase.
[2016-09-27 19:44:15,173] [DEBUG] Finished epoch 0. (Avg. training loss: 1.20308633645)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1076: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1076: UndefinedMetricWarning: F-score is ill-def

In [ ]:
# model.wv_model.vocab
print len(model.test_one_hot.get_key_ordering())
print len(model.one_hot.get_key_ordering())

# model.one_hot.get_key_ordering().where( model.w )

import numpy as np
diffvecs = np.linalg.inv( model.w.T.dot(model.w) ).dot(model.w.T).dot(model.Cwd)


In [ ]:
logger.info("Starting evaluation phase.")
evaluate(sess, model, fetches, feed_dict, truth)